In [ ]:
import numpy as np
import itertools

In [ ]:
test_text = """029A
980A
179A
456A
379A"""

In [ ]:
with open("input.txt") as f:
    data_input = f.read()

In [ ]:
def add_tuple(a, b):
    return(a[0] + b[0], a[1] + b[1])

def sub_tupple(a, b):
    return(a[0] - b[0], a[1] - b[1])
    

In [ ]:
class Pad:
    def __init__(self):
        board_str = \
        """
        789
        456
        123
        _0A
        """
        self.board = np.array([[s for s in line.strip()] for line in board_str.strip().split("\n")])
        self.pos = self.get_pos("A")
        self.row_first = True
    
    def get_pos(self, button):
        pos_np = np.where(self.board == button)
        return tuple(map(int, (pos_np[0][0], pos_np[1][0])))
    
    def get_row_command(delf, dp):
        return_command = ""
        if dp[0] > 0:
            return_command += "v" * abs(dp[0])
        else:
            return_command += "^" * abs(dp[0])
        return return_command
  
      
    def get_col_command(delf, dp):
        return_command = ""
        if dp[1] > 0:
            return_command += ">" * abs(dp[1])
        else:
            return_command += "<" * abs(dp[1])
        return return_command  

    def get_commands(self, dp):
        return_command = ""
        if self.row_first:
            if self.board[(dp[0] + self.pos[0], self.pos[1])] != "_":
                return_command += self.get_row_command(dp)
                return_command += self.get_col_command(dp)
            else:
                return_command += self.get_col_command(dp)
                return_command += self.get_row_command(dp)
        else:
            if self.board[(self.pos[0], dp[1] + self.pos[1])] != "_":
                return_command += self.get_col_command(dp)
                return_command += self.get_row_command(dp)
            else:
                return_command += self.get_row_command(dp)
                return_command += self.get_col_command(dp)
        return_command += "A"
        return return_command

    def push_button(self, button):
        button_pos = self.get_pos(button)
        dp = sub_tupple(button_pos, self.pos)
        return_command = self.get_commands(dp)
        self.pos = button_pos
        return return_command
    
    def get_all_combinations(self, button):
        combinations = []
        starting_pas = self.pos
        row_first = self.row_first
        self.row_first = True
        combinations.append(self.push_button(button))
        self.pos = starting_pas
        self.row_first = False
        combinations.append(self.push_button(button))
        self.row_first = row_first
        return combinations
    
    def parse_instruction(self, instr):
        ret = ""
        for button in instr: 
            ret += self.push_button(button)
        return ret
        

In [ ]:
class Robot(Pad):
    def __init__(self):
        super(Robot, self).__init__()
        board_str = \
        """
        _^A
        <v>
        """
        self.board = np.array([[s for s in line.strip()] for line in board_str.strip().split("\n")])
        self.pos = self.get_pos("A")
        self.row_first_mutable = True

    def push_button(self, button):
        if not self.row_first_mutable:
            return super().push_button(button)
        row_first = self.row_first
        button_pos = self.get_pos(button)
        dp = sub_tupple(button_pos, self.pos)
        if dp[1] > 0:
            self.row_first = True
        else:
            self.row_first = False
        ret = super().push_button(button)
        self.row_first = row_first
        return ret


In [ ]:
def parse_input(text, robots_cnt):
    pad = Pad()
    robots = [Robot() for _ in range(robots_cnt)]
    res = ''
    for i in text:
        pad_combs = pad.get_all_combinations(i)
        min_res_len = float('inf')
        res_i = ""
        for pad_comb in pad_combs:
            robot_instr = pad_comb
            for robot in robots:
                robot_instr = robot.parse_instruction(robot_instr)
            if min_res_len > len(robot_instr):
                min_res_len = len(robot_instr)
                res_i = robot_instr
        res += res_i
    return res

In [ ]:
def split_instruction(instr: str):
    res = []
    find = instr.find("A")
    while find != -1:
        res.append(instr[:find + 1])
        instr = instr[find + 1:]
        find = instr.find("A")
    return res

def get_transform_dict():
    start_pos = [i for i in"<>^vA"]
    end_pos = start_pos.copy()
    start_end_pairs = list(itertools.product(start_pos, end_pos))
    start_end_pairs
    res_list = []
    for pair in start_end_pairs:
        robot = Robot()
        robot.pos = robot.get_pos(pair[0])
        res = robot.push_button(pair[1])
        res_list.append(res)

    res_set = set(res_list)
    out_dict = dict()
    for item in res_set:
        robot = Robot()
        robot.pos = robot.get_pos("A")
        res = robot.parse_instruction(item)
        out_dict[item] = res
    out_dict
    out_list_dict = dict()
    for k, v in out_dict.items():
        out_list_dict[k] = split_instruction(v)
    return out_list_dict

def get_total_len(level_dict: dict):
    res = 0
    for k, v in level_dict.items():
        res += len(k) * v
    return res

def parse_level(res: str):
    res_split = split_instruction(res)
    transform_dict = get_transform_dict()
    level_instr = dict.fromkeys(transform_dict.keys(), 0)
    for instr in res_split:
        level_instr[instr] += 1
    return level_instr

def get_next_level(level_dict: dict, transform_dict: dict): 
    next_level_instr = dict.fromkeys(level_dict.keys(), 0)
    for k, v in level_dict.items():
        if v == 0:
            continue
        next_items = transform_dict[k]
        for item in next_items:
            next_level_instr[item] += v
    return next_level_instr

### Part one

In [ ]:
control_sum = 0
inputs = data_input.strip().split("\n")

for instr in inputs:
    res = parse_input(instr, 2)
    val = int(instr[:-1])
    val *= len(res)
    control_sum += val
print(control_sum)

### Part two

In [ ]:
inputs = data_input.strip().split("\n")
control_sum = 0
for text in inputs:
    res = parse_input(text, 2)
    val = int(text[:-1])

    level_instr = parse_level(res)
    transform_dict = get_transform_dict()
    for _ in range(25 - 2):
        next_level = get_next_level(level_instr, transform_dict)
        level_instr = next_level
    
    val *= get_total_len(next_level)
    control_sum += val

control_sum